<a href="https://colab.research.google.com/github/PadawanXXVI/spotify-data-storytelling/blob/main/spotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎓 Spotify Data Storytelling — Semana 2  
## Fase 2 — Coleta, Limpeza e Transformação de Dados

**Curso:** Ciência de Dados — Faculdade de Tecnologia e Inovação Senac DF  
**Disciplina:** Projeto Interdisciplinar  
**Professor:** Prof. Alexander Barreto  
**Autor:** Anderson de Matos Guimarães  

---

## 📘 Apresentação

Este notebook documenta a **segunda fase do projeto Spotify Data Storytelling**, que tem como objetivo:
1. Conectar-se à **API oficial do Spotify** por meio de autenticação segura;
2. Realizar a **coleta automatizada de dados musicais** (gêneros e faixas populares no Brasil);
3. Efetuar a **limpeza, padronização e transformação dos dados**;
4. Gerar o dataset tratado que servirá de base para a **análise visual e storytelling (Semana 3)**.

---

## 🌍 Relevância Acadêmica e Comercial

O projeto une **fundamentos de Engenharia de Dados, Estatística e Visualização Analítica** com foco em:
- Compreender **padrões de consumo musical no Brasil**;  
- Aplicar metodologias **CRISP-DM e boas práticas reprodutíveis**;  
- Gerar valor **comercial e científico**, permitindo que dados musicais fundamentem decisões estratégicas em curadoria, marketing e recomendação.

---


In [12]:
# ⚙️ Instalação das dependências necessárias
!pip install spotipy pandas numpy matplotlib seaborn plotly

# 📦 Importação das bibliotecas
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from google.colab import userdata

print("✅ Bibliotecas instaladas e importadas com sucesso!")


✅ Bibliotecas instaladas e importadas com sucesso!


In [13]:
# 🔑 Autenticação via variáveis seguras do Google Colab
client_id = userdata.get("SPOTIPY_CLIENT_ID")
client_secret = userdata.get("SPOTIPY_CLIENT_SECRET")

if not client_id or not client_secret:
    raise ValueError("❌ Erro: Variáveis de autenticação não encontradas no ambiente.")

auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

print("🔐 Autenticação com a API Spotify concluída com sucesso!")


🔐 Autenticação com a API Spotify concluída com sucesso!


In [14]:
# 🎧 Coleta de faixas populares no Brasil por gênero musical

generos_validos = [
    "sertanejo", "sertanejo universitário", "forró", "mpb", "bossa nova",
    "samba", "pagode", "funk", "rap", "trap", "pop", "rock",
    "indie", "gospel", "eletrônica", "country", "reggae", "jazz", "blues"
]

print(f"🎯 Total de gêneros para coleta: {len(generos_validos)}\n")

all_tracks = []
resumo = []

for genero in generos_validos:
    print(f"🎶 Coletando faixas do gênero: {genero}")
    try:
        results = sp.search(q=f"genre:{genero}", type="track", limit=50, market="BR")
        tracks = results["tracks"]["items"]

        if len(tracks) == 0:
            print(f"⚠️ Nenhum resultado para o gênero: {genero}")
            continue

        for item in tracks:
            all_tracks.append({
                "genero": genero,
                "faixa": item["name"],
                "artista": item["artists"][0]["name"],
                "album": item["album"]["name"],
                "data_lancamento": item["album"]["release_date"],
                "popularidade": item["popularity"],
                "duracao_ms": item["duration_ms"]
            })

        resumo.append({"genero": genero, "qtd_faixas": len(tracks)})

    except Exception as e:
        print(f"⚠️ Erro ao coletar o gênero {genero}: {e}")

# ✅ Criação dos DataFrames
df_raw = pd.DataFrame(all_tracks)
df_resumo = pd.DataFrame(resumo).sort_values("qtd_faixas", ascending=False)

print(f"\n✅ Coleta concluída: {len(df_raw)} faixas coletadas.")
display(df_resumo)


🎯 Total de gêneros para coleta: 19

🎶 Coletando faixas do gênero: sertanejo
🎶 Coletando faixas do gênero: sertanejo universitário
🎶 Coletando faixas do gênero: forró
🎶 Coletando faixas do gênero: mpb
🎶 Coletando faixas do gênero: bossa nova
🎶 Coletando faixas do gênero: samba
🎶 Coletando faixas do gênero: pagode
🎶 Coletando faixas do gênero: funk
🎶 Coletando faixas do gênero: rap
🎶 Coletando faixas do gênero: trap
🎶 Coletando faixas do gênero: pop
🎶 Coletando faixas do gênero: rock
🎶 Coletando faixas do gênero: indie
🎶 Coletando faixas do gênero: gospel
🎶 Coletando faixas do gênero: eletrônica
🎶 Coletando faixas do gênero: country
🎶 Coletando faixas do gênero: reggae
🎶 Coletando faixas do gênero: jazz
🎶 Coletando faixas do gênero: blues

✅ Coleta concluída: 950 faixas coletadas.


,genero,qtd_faixas
0,sertanejo,50
1,sertanejo universitário,50
2,forró,50
3,mpb,50
4,bossa nova,50
5,samba,50
6,pagode,50
7,funk,50
8,rap,50
9,trap,50


In [15]:
# 💾 Exporta dataset bruto para análise posterior
os.makedirs("/content/data", exist_ok=True)
df_raw.to_csv("/content/data/spotify_raw.csv", index=False)
print("💾 Dataset bruto salvo em /content/data/spotify_raw.csv")


💾 Dataset bruto salvo em /content/data/spotify_raw.csv


In [16]:
# 📊 Diagnóstico inicial
print("🔍 Estrutura inicial do dataset bruto:\n")
print(f"Dimensões: {df_raw.shape[0]} linhas e {df_raw.shape[1]} colunas\n")
df_raw.info()

print("\n📈 Estatísticas descritivas iniciais:")
display(df_raw.describe(include='all'))


🔍 Estrutura inicial do dataset bruto:

Dimensões: 950 linhas e 7 colunas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   genero           950 non-null    object
 1   faixa            950 non-null    object
 2   artista          950 non-null    object
 3   album            950 non-null    object
 4   data_lancamento  950 non-null    object
 5   popularidade     950 non-null    int64 
 6   duracao_ms       950 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 52.1+ KB

📈 Estatísticas descritivas iniciais:


,genero,faixa,artista,album,data_lancamento,popularidade,duracao_ms
count,950,950,950,950,950,950.000000,950.000000
unique,19,833,444,750,638,NaN,NaN
top,sertanejo,"Augusta, angélica e consolação",Tom Zé,The Life of a Showgirl,2024-05-28,NaN,NaN
freq,50,7,42,11,13,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,27.001053,212332.618947
std,NaN,NaN,NaN,NaN,NaN,36.160149,74401.422298
min,NaN,NaN,NaN,NaN,NaN,0.000000,15482.000000
25%,NaN,NaN,NaN,NaN,NaN,0.000000,170674.000000
50%,NaN,NaN,NaN,NaN,NaN,0.000000,205099.500000
75%,NaN,NaN,NaN,NaN,NaN,68.000000,242925.500000


In [17]:
# 🧹 Limpeza e padronização dos dados
df = df_raw.copy()

# Remoção de duplicados
df.drop_duplicates(subset=["faixa", "artista", "album"], inplace=True)

# Conversão de tipos
df["data_lancamento"] = pd.to_datetime(df["data_lancamento"], errors="coerce")
df["popularidade"] = pd.to_numeric(df["popularidade"], errors="coerce")

# Padronização textual
for col in ["faixa", "artista", "album", "genero"]:
    df[col] = df[col].astype(str).str.strip().str.title()

print("🧽 Limpeza concluída. Dataset padronizado com sucesso!")


🧽 Limpeza concluída. Dataset padronizado com sucesso!


In [18]:

# 🩺 Diagnóstico pós-tratamento
print("\n📋 Estrutura final do dataset tratado:")
df.info()

print("\n📈 Estatísticas descritivas após tratamento:")
display(df.describe())

print("\n🩶 Nulos restantes:")
display(df.isnull().sum())



📋 Estrutura final do dataset tratado:
<class 'pandas.core.frame.DataFrame'>
Index: 890 entries, 0 to 941
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   genero           890 non-null    object        
 1   faixa            890 non-null    object        
 2   artista          890 non-null    object        
 3   album            890 non-null    object        
 4   data_lancamento  845 non-null    datetime64[ns]
 5   popularidade     890 non-null    int64         
 6   duracao_ms       890 non-null    int64         
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 55.6+ KB

📈 Estatísticas descritivas após tratamento:


,data_lancamento,popularidade,duracao_ms
count,845,890.000000,890.00000
mean,2013-04-26 07:06:02.130177536,26.449438,212694.71236
min,1905-06-19 00:00:00,0.000000,15482.00000
25%,2009-06-01 00:00:00,0.000000,170059.25000
50%,2019-05-24 00:00:00,0.000000,205825.00000
75%,2022-07-18 00:00:00,68.000000,243081.50000
max,2025-10-31 00:00:00,100.000000,602394.00000
std,NaN,35.963271,75073.78059



🩶 Nulos restantes:


,0
genero,0
faixa,0
artista,0
album,0
data_lancamento,45
popularidade,0
duracao_ms,0


In [19]:
# 🎨 Visualização exploratória (corrigida e validada)
import matplotlib.pyplot as plt
import seaborn as sns

if "popularidade_norm" in df.columns and "duracao_min" in df.columns:
    plt.figure(figsize=(10,5))
    sns.histplot(df["popularidade_norm"], bins=20, kde=True, color="royalblue")
    plt.title("Distribuição da Popularidade Normalizada")
    plt.xlabel("Popularidade Normalizada (0–1)")
    plt.ylabel("Frequência")
    plt.show()

    plt.figure(figsize=(10,5))
    sns.scatterplot(x="duracao_min", y="popularidade_norm", data=df, alpha=0.6, color="darkgreen")
    plt.title("Duração (min) × Popularidade Normalizada")
    plt.xlabel("Duração (minutos)")
    plt.ylabel("Popularidade Normalizada")
    plt.show()
else:
    print("⚠️ As colunas derivadas ainda não estão disponíveis para visualização.")


⚠️ As colunas derivadas ainda não estão disponíveis para visualização.


In [20]:
# 💾 Exportação do dataset tratado
output_path = "/content/data/spotify_semana2_tratado.csv"
df.to_csv(output_path, index=False)

print(f"💾 Dataset tratado exportado com sucesso: {output_path}")


💾 Dataset tratado exportado com sucesso: /content/data/spotify_semana2_tratado.csv
